In [3]:
from jupyter_client import find_connection_file
connection_file = find_connection_file()
print(connection_file)

/home/yonatan/.local/share/jupyter/runtime/kernel-v2-4434y6DBCN3sR1qi.json


In [4]:
# Plotting related
import os
import sys

sys.tracebacklimit = 0

import seaborn as sns

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio


# Scikit-learn related imports
import numpy as np
import pandas as pd

pd.set_option("display.float_format", "{:.2f}".format)
pd.set_option("display.max_colwidth", None)
pd.options.mode.copy_on_write = True

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint, ttest_ind
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_auc_score, precision_recall_curve
from sklearn.cluster import KMeans
from math import log

import importlib
# Constants for this project
import constants
from constants import *


import papermill as pm
import shutil
shutil.copy('constants.py', 'export/')

# Change the current working directory
os.chdir("/home/yonatan/Documents/projects/vaccines/code")

In [5]:
bAdjustMFC

True

In [6]:
# Define auxilary functions
def get_dir_by_name(dir_name):
    # Define the starting directory
    current_dir = os.getcwd()

    # Traverse up the directory tree until we find a directory named "data"
    while current_dir != "/":
        if dir_name in os.listdir(current_dir):
            data_dir = os.path.join(current_dir, dir_name)
            return data_dir
        current_dir = os.path.dirname(current_dir)
    else:
        print(f"Directory {dir_name} not found in the parent directories.")
        raise (Exception())


def remove_duplicate_accessions(dataset, immage_col, uid_col):
    '''Sometimes there are multiple geo_accession numbers, like in GSE48018.SDY1276.
    Average the IMMAGE, since all else is the same'''
    first_uid = dataset.iloc[0][uid_col]
    accessions = dataset[dataset[uid_col] == first_uid]["geo_accession"].unique()
    if len(accessions) > 1:
        # print(f"Multiple accession detected, Collapsing by averaging on IMMAGE value")
        dataset = dataset.groupby(uid_col, as_index=False).agg({immage_col: "mean", **{col: "first" for col in dataset.columns if col not in [uid_col, immage_col]},})

    accessions = dataset[dataset[uid_col] == first_uid]["geo_accession"].unique()
    assert len(accessions) == 1

    return dataset

def get_threshold_from_probability(prob, intercept, slope):
    return -1 * (log(1 / prob - 1) + intercept) / slope

def calc_and_plot_prob_threshold(data, classifier, precision, recall, thresholds, prob_column, features=""):
    AUC = auc(recall, precision)
    intercept = classifier.intercept_[0]
    slope = classifier.coef_[0][0]

    naive_classification_precision = data["y"].mean()

    # Identifying the optimal threshold (maximal F1 score)
    beta = 0.7
    F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
    optimal_idx = np.nanargmax(F_scores)
    prob_threshold = thresholds[optimal_idx]
    score = F_scores[optimal_idx]

    # Calculate the cutoff value
    feature_threshold = get_threshold_from_probability(
        prob_threshold, intercept=intercept, slope=slope
    )

    return (score, prob_threshold, feature_threshold, AUC)


def get_classifier_stats_prob(data, prob_column, prob_threshold):
    # Global measures (entire dataset)
    optimal_pred = data[prob_column].apply(lambda x: 1 if x >= prob_threshold else 0)
    test_accuracy = accuracy_score(data["y"], optimal_pred)
    # Performance above the prob_threshold
    y_over_thr = data.loc[data[prob_column] >= prob_threshold, ["y"]]
    non_response_rate_over_thr = y_over_thr.mean().y
    y_under_thr = data.loc[data[prob_column] < prob_threshold, ["y"]]
    non_response_rate_under_thr = y_under_thr.mean().y
    return non_response_rate_over_thr, non_response_rate_under_thr

In [7]:
def analyze_dataset(dataset, P):
    # These parameters are overridden by papermill
    dataset_name = dataset[dataset_col].iloc[0]
    bAdjustMFC = P["bAdjustMFC"]
    strain_index = P["strain_index"]
    day = P["day"]
    day0  = P["day0"]
    dayMFC = P["dayMFC"]

    # Discard seroprotected subjects based on HAI > 40 threshold)
    if bDiscardSeroprotected:
        day0_mask = dataset[day_col] == day0
        threshold_mask = dataset[response_col]> HAI_threshold

        # Get a list of all protected patients
        serprotected_subjects = dataset.loc[(day0_mask) & (threshold_mask)][uid_col].unique()
        # keep only patients not in the serprotected_subjects list
        dataset = dataset.loc[~dataset[uid_col].isin(serprotected_subjects)]
        subjects_left = dataset[uid_col].unique()
        print(f"Discarding {len(serprotected_subjects)} seroprotected subjects")
        print(f"Subjects left: N={len(subjects_left)}")

    # Pivot the dataset such that different days' samples appear in their own columns, witn NaN where there are missing samples
    t = dataset[[dataset_col, uid_col, age_col, immage_col,accesion_col , day_col, response_col]]
    pivot_t = t.pivot_table(index=uid_col, columns=day_col, values=response_col, aggfunc='first')
    age_t = dataset[['uid', 'Age']].drop_duplicates()

    # Average IMMAGE values across geo_accessions (if they exist) and merge
    immage_t = t.groupby('uid')[immage_col].mean()
    tmp_t = age_t.merge(immage_t, on='uid', how='left').drop_duplicates()
    pivot_t = tmp_t.merge(pivot_t, on='uid', how='left')

    # Reset index to make uid a column again
    pivot_t.reset_index(inplace=True, drop=True)

    # Remove the name of the columns and index name
    pivot_t.columns.name = None
    pivot_t.index.name = None

    # TODO complete generating MFC for this dataset
    # if dataset_name == 'SDY296' or dataset_name == 'GSE48023.SDY1276':
    #     dataset = dataset.loc[(~pivot_t[day0].isna()) & (~pivot_t["FC.HAI"].isna())]
    #     pivot_t[dayMFC] = pivot_t[day] / pivot_t[day0]

    # Currently only used by AdjustMFC branch. TODO: convert the "regular" branch to use it too
    pivot_dataset = pivot_t

    # Use adjusted MFC (HAI) as per John Tsang
    cluster_col = day0
    if bAdjustMFC:
        # raise(Exception("Did you mean to use adjMFC?"))
        dataset = pivot_dataset[[uid_col, immage_col, age_col, day0, dayMFC]]
        print("Preprocessing dataset, computing adjusted MFC (HAI)")
        dataset = dataset.loc[(~pivot_t[day0].isna()) & (~pivot_t[dayMFC].isna())]

        mean = dataset[day0].mean()
        std = dataset[day0].std()
        threshold = 3 * std
        dataset = dataset[(dataset[day0] >= mean - threshold) & (dataset[day0] <= mean + threshold)]

        # Bin subjects into 2-3 bins using k-means clustering
        kmeans = KMeans(n_clusters=3, random_state=0)
        dataset['Cluster'] = kmeans.fit_predict(dataset[[cluster_col]])

        def normalize(x):
            return (x - x.median()) / x.std()

        # Normalize the MFC within each bin to obtain the adjMFC
        dataset['adjMFC'] = dataset.groupby('Cluster')[dayMFC].transform(normalize)

        # Take relevant columns only
        data = dataset[[immage_col, 'adjMFC', age_col, cluster_col, "Cluster"]].rename(columns={'adjMFC': response_col}).dropna()
        # data.groupby("Cluster").count()
        strain = "Influenza"
        strains = "Influenza"

    else: # bAdjustMFC == False
        # If not computing adjMFC, take a specific strain from the given post-vaccine day & assay
        dayMFC_mask = dataset[day_col] == day
        dataset = dataset.loc[(dayMFC_mask)].reset_index(drop=True)

        # Somtimes there are multiple strains - so multiple rows per day
        strains = dataset[strain_col].unique()
        if len(strains) > 1:
            dataset = dataset.loc[dataset[strain_col] == strains[strain_index]].reset_index(drop=True)

        strains_t = dataset[strain_col].unique()
        assert len(strains_t) == 1
        strain = strains_t[0]

        dataset = remove_duplicate_accessions(dataset, immage_col, uid_col)

        # Take relevant columns only
        data = dataset[[immage_col, response_col, age_col]]

    # Keep older subjects only, since that's what's actually more interesting, and may show IMMAGE's advantage
    if bOlderOnly == True:
        young_subjects = data.loc[data[age_col] < age_threshlod]
        data = data.loc[data[age_col] >= age_threshlod]
        if len(data) == 0:
            raise(Exception("No subjects over the age of {age_threshlod}. Exiting."))
        print(f"Discarding {len(young_subjects)} seroprotected subjects")
        print(f"Subjects left: N={len(data)}")


    #### Dataset & Strain info
    age_restrict_str = f", Subjects over the age of {age_threshlod}" if bOlderOnly else ""
    day_str = "Adjusted MFC" if bAdjustMFC else f"day: {day}"

    print(f"""### Analysis for dataset: {dataset_name}, strain: {strain}, {day_str}{age_restrict_str}""")

    data.reset_index(inplace=True, drop=True)

    # Get a boolean map of sub and super threshold values
    low_response_thr = data[[response_col]].quantile(q=0.3).item()

    # Generate labels
    # Note that we define y=1 for all responses <= 30th percentile (and not <)
    # Also note that we defined y=1 as *non* responders, since later on that's what we'll care about detecting
    data["y"] = data[response_col].apply(lambda x: 1 if x <= low_response_thr else 0)

    # Add a text label for plot legends
    data["Label text"] = data["y"].apply(lambda x: "Responders" if x == 0 else "Non-Responders")


    # Classifying with logistic regression - fit on the entire dataset
    log_regress_immage = LogisticRegression()
    log_regress_age = LogisticRegression()
    log_regress_combined = LogisticRegression()

    # Train a classifier based on immage and on age for comparison
    log_regress_immage.fit(data[[immage_col]], data["y"])
    log_regress_age.fit(data[[age_col]], data["y"])
    log_regress_combined.fit(data[[immage_col, age_col]], data["y"])

    non_responder_col = "p_non_responder"
    non_responder_col_age = "p_non_responder_age"
    non_responder_col_combined = "p_non_responder_combined"

    proba = pd.DataFrame(log_regress_immage.predict_proba(data[[immage_col]]))
    data[non_responder_col] = proba[1]
    proba = pd.DataFrame(log_regress_age.predict_proba(data[[age_col]]))
    data[non_responder_col_age] = proba[1]
    proba = pd.DataFrame(log_regress_combined.predict_proba(data[[immage_col, age_col]]))
    data[non_responder_col_combined] = proba[1]

    # #### Thresholding based on logistic regression probabilties
    # #### IMMAGE-based classification
    # Run for immage and age to compare
    # IMMAGE
    precision, recall, thresholds = precision_recall_curve(data["y"], data[non_responder_col])
    immage_score, prob_threshold, immage_threshold, immage_auc = calc_and_plot_prob_threshold(
        data, log_regress_immage, precision, recall, thresholds, non_responder_col, features=[immage_col]
    )
    non_response_rate_over_thr, non_response_rate_under_thr = get_classifier_stats_prob(
        data, non_responder_col, prob_threshold
    )


    # #### Age-based classification
    # Age
    precision, recall, thresholds = precision_recall_curve(data["y"], data[non_responder_col_age])
    age_score, prob_threshold_age, age_threshold, age_auc = calc_and_plot_prob_threshold(
        data, log_regress_age, precision, recall, thresholds, non_responder_col_age, features=[age_col]
    )
    age_non_response_rate_over_thr, age_non_response_rate_under_thr = get_classifier_stats_prob(
        data, non_responder_col_age, prob_threshold_age
    )


    # #### Age & IMMAGE combined
    # Combined
    precision, recall, thresholds = precision_recall_curve(data["y"], data[non_responder_col_combined])
    combined_score, prob_threshold_combined, _, combined_auc = calc_and_plot_prob_threshold(
        data, log_regress_combined, precision, recall, thresholds, non_responder_col_combined, features=[immage_col, age_col]
    )
    combined_non_response_rate_over_thr, combined_non_response_rate_under_thr = (
        get_classifier_stats_prob(data, non_responder_col_combined, prob_threshold_combined)
    )


    # #### Comparison of using the different features
    summary_dict = {
    ("F score", "IMMAGE"): [immage_score],
    ("F score", "Age"): [age_score],
    ("F score", "Multivariate"): [combined_score],
    ("NR rate over threshold", "IMMAGE"): [non_response_rate_over_thr],
    ("NR rate over threshold", "Age"): [age_non_response_rate_over_thr],
    ("NR rate over threshold", "Multivariate"): [combined_non_response_rate_over_thr],
    ("NR rate under threshold", "IMMAGE"): [non_response_rate_under_thr],
    ("NR rate under threshold", "Age"): [age_non_response_rate_under_thr],
    ("NR rate under threshold", "Multivariate"): [combined_non_response_rate_under_thr],
}

    # Create a MultiIndex
    multi_index = pd.MultiIndex.from_product([["F score", "NR rate over threshold", "NR rate under threshold"], ["IMMAGE", "Age", "Multivariate"]])

    # Create the DataFrame
    summary = pd.DataFrame(summary_dict, columns=multi_index)
    summary["Composite", "IMMAGE"] = summary[[("F score", "IMMAGE"), ("NR rate over threshold", "IMMAGE")]].mean(axis=1)
    summary["Composite", "Age"] = summary[[("F score", "Age"), ("NR rate over threshold", "Age")]].mean(axis=1)
    summary["Composite", "Multivariate"] = summary[[("F score", "Multivariate"), ("NR rate over threshold", "Multivariate")]].mean(axis=1)
    # print(summary.to_string(index=False))

    return summary

In [8]:
def analyze_all_datasets(datasets, metadata):
    accumulated_results = pd.DataFrame()
    for bAdjustMFC in [True, False]:
        for dataset_name in metadata[dataset_col].unique():
                curr_metadata = metadata.loc[metadata[dataset_col] == dataset_name]
                dataset = datasets.loc[datasets[dataset_col] == dataset_name]
                print(dataset_name)
                days = curr_metadata["Days"].iloc[0]
                for day in days:
                        adjMFC_str = "AdjMFC" if bAdjustMFC else f"day: {day}"
                        strains = dataset.loc[dataset[day_col] == day][strain_col].unique()
                        # print(strains)
                        for strain_index in range(len(strains)):
                                strain_name = strains[strain_index].replace("/", "_").replace(" ", "_")
                                # print(f'exporting {dataset_name}, strain no. {strain_index}: {strain_name}, day: {day}')
                                # Define parameters for curr_metadata and strain
                                P = {
                                    "bAdjustMFC": bAdjustMFC,
                                    "dataset_name": dataset_name,
                                    "strain_index": strain_index,
                                    "day":  day,
                                    "day0":  curr_metadata["Day0"].iloc[0],
                                    "dayMFC":  curr_metadata["DayMFC"].iloc[0],
                                }
                                try:
                                    tmp_dict = {
                                        dataset_col: dataset_name,
                                        strain_col: strain_name,
                                        strain_index_col: strain_index,
                                        day_col: "AdjMFC" if bAdjustMFC else f"{day}",
                                        "bAdjustMFC" :  bAdjustMFC,
                                    }
                                    # initialize the row for this database with some metadata
                                    row  = pd.DataFrame([tmp_dict])
                                    row = pd.concat([row, analyze_dataset(dataset, P)], axis=1) # Concat along lines, adding new columns
                                    accumulated_results = pd.concat([accumulated_results, row], ignore_index=True) # Concat along columns, adding new lines
                                except:
                                        print (f"******\nCaught exception when runnnig {dataset_name}\n******\n")
    return accumulated_results



In [29]:
# def main():

# Read in Data and drop missing values
data_dir = get_dir_by_name("data")
df = pd.read_csv(os.path.join(data_dir, "../data/all_vaccines.csv"))
datasets = df.dropna(subset=[immage_col, age_col, dataset_col, uid_col, day_col, response_col])
dataset_names = datasets[dataset_col].unique()

if bInfluenza:
    # Get the info for all influenza datasets, excluding some.
    metadata = pd.DataFrame(influenza_dicts)
    dataset_names = metadata[dataset_col].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(exclude_datasets))
    datasets = datasets.loc[datasets["Dataset"].isin(dataset_names)]
    print("Working with Influenza datasets only")


# # Narrow to a specific datset
# dataset_name = "GSE41080.SDY212"
# # Filter data
# name_mask = datasets[dataset_col] == dataset_name
# dataset = datasets.loc[name_mask].reset_index(drop=True)

# # Filter metadata
# name_mask = metadata[dataset_col] == dataset_name
# metadata = metadata.loc[name_mask].reset_index(drop=True)

# P = {
#     "dataset_name": dataset_name,
#     "strain_index": 0,
#     "day":  metadata["Days"].iloc[0][0],
#     "day0":  metadata["Day0"].iloc[0],
#     "dayMFC":  metadata["DayMFC"].iloc[0],
# }

# This will print a sort of log of the different analyses
results = analyze_all_datasets(datasets, metadata)

# Get all the analyses that look somewhat promising based on the composite (F1 and over-threshold rate) metric
score_mask = (results["Composite", "IMMAGE"] >  results["Composite", "Age"]) | (results["Composite", "Multivariate"] >  results["Composite", "Age"])
cols_to_access = [
    dataset_col,
    strain_col,
    strain_index_col,
    day_col,
]

# Dynamically add all sub-columns for 'Composite'
composite_columns = [col for col in results.columns if col[0] == 'Composite']
cols_to_access.extend(composite_columns)
results = results.loc[score_mask]

Working with Influenza datasets only
GSE41080.SDY212
Discarding 45 seroprotected subjects
Subjects left: N=44
Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 10 seroprotected subjects
Subjects left: N=34
### Analysis for dataset: GSE41080.SDY212, strain: Influenza, Adjusted MFC, Subjects over the age of 60
Discarding 45 seroprotected subjects
Subjects left: N=44
Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 10 seroprotected subjects
Subjects left: N=34
### Analysis for dataset: GSE41080.SDY212, strain: Influenza, Adjusted MFC, Subjects over the age of 60
Discarding 45 seroprotected subjects
Subjects left: N=44
Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 10 seroprotected subjects
Subjects left: N=34
### Analysis for dataset: GSE41080.SDY212, strain: Influenza, Adjusted MFC, Subjects over the age of 60
Discarding 45 seroprotected subjects
Subjects left: N=44
Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 10 seroprotected 

/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/home/yonatan/miniforge3/envs/vaccines/lib/python3

Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 10 seroprotected subjects
Subjects left: N=34
### Analysis for dataset: GSE41080.SDY212, strain: Influenza, Adjusted MFC, Subjects over the age of 60
GSE48018.SDY1276
Discarding 0 seroprotected subjects
Subjects left: N=115
Preprocessing dataset, computing adjusted MFC (HAI)
******
Caught exception when runnnig GSE48018.SDY1276
******

Discarding 0 seroprotected subjects
Subjects left: N=115
Preprocessing dataset, computing adjusted MFC (HAI)
******
Caught exception when runnnig GSE48018.SDY1276
******

Discarding 0 seroprotected subjects
Subjects left: N=115
Preprocessing dataset, computing adjusted MFC (HAI)
******
Caught exception when runnnig GSE48018.SDY1276
******

Discarding 0 seroprotected subjects
Subjects left: N=115
Preprocessing dataset, computing adjusted MFC (HAI)
******
Caught exception when runnnig GSE48018.SDY1276
******

Discarding 0 seroprotected subjects
Subjects left: N=115
Preprocessing dataset, comput

/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to sup

Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 9 seroprotected subjects
Subjects left: N=14
### Analysis for dataset: GSE59654.SDY404, strain: Influenza, Adjusted MFC, Subjects over the age of 60
Discarding 16 seroprotected subjects
Subjects left: N=23
Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 9 seroprotected subjects
Subjects left: N=14
### Analysis for dataset: GSE59654.SDY404, strain: Influenza, Adjusted MFC, Subjects over the age of 60
Discarding 16 seroprotected subjects
Subjects left: N=23
Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 9 seroprotected subjects
Subjects left: N=14
### Analysis for dataset: GSE59654.SDY404, strain: Influenza, Adjusted MFC, Subjects over the age of 60
Discarding 16 seroprotected subjects
Subjects left: N=23
Preprocessing dataset, computing adjusted MFC (HAI)
Discarding 9 seroprotected subjects
Subjects left: N=14
### Analysis for dataset: GSE59654.SDY404, strain: Influenza, Adjusted MFC, Subjects 

...

/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)


Discarding 16 seroprotected subjects
Subjects left: N=23
Discarding 9 seroprotected subjects
Subjects left: N=14
### Analysis for dataset: GSE59654.SDY404, strain: A/California/7/2009, day: FC.HAI, Subjects over the age of 60
GSE59743.SDY400
Discarding 8 seroprotected subjects
Subjects left: N=22
Discarding 7 seroprotected subjects
Subjects left: N=13
### Analysis for dataset: GSE59743.SDY400, strain: B/Wisonsin/01/2010, day: FC.HAI, Subjects over the age of 60
Discarding 8 seroprotected subjects
Subjects left: N=22
Discarding 7 seroprotected subjects
Subjects left: N=13
### Analysis for dataset: GSE59743.SDY400, strain: A/Victoria/361/2011, day: FC.HAI, Subjects over the age of 60
Discarding 8 seroprotected subjects
Subjects left: N=22
Discarding 7 seroprotected subjects
Subjects left: N=13
### Analysis for dataset: GSE59743.SDY400, strain: A/California/7/2009, day: FC.HAI, Subjects over the age of 60
Discarding 8 seroprotected subjects
Subjects left: N=22
Discarding 7 seroprotected s

/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))

SDY67
Discarding 127 seroprotected subjects
Subjects left: N=32
Discarding 19 seroprotected subjects
Subjects left: N=13
### Analysis for dataset: SDY67, strain: A/California/7/2009, day: FC.D28.HAI, Subjects over the age of 60
Discarding 127 seroprotected subjects
Subjects left: N=32
Discarding 19 seroprotected subjects
Subjects left: N=13
### Analysis for dataset: SDY67, strain: A/Perth/16/2009, day: FC.D28.HAI, Subjects over the age of 60
Discarding 127 seroprotected subjects
Subjects left: N=32
Discarding 19 seroprotected subjects
Subjects left: N=13
### Analysis for dataset: SDY67, strain: A/California/7/2009, day: HAI.D28, Subjects over the age of 60
Discarding 127 seroprotected subjects
Subjects left: N=32
Discarding 19 seroprotected subjects
Subjects left: N=13
### Analysis for dataset: SDY67, strain: A/Perth/16/2009, day: HAI.D28, Subjects over the age of 60
GSE59635.SDY63
Discarding 7 seroprotected subjects
Subjects left: N=11
Discarding 7 seroprotected subjects
Subjects left

/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)
/tmp/ipykernel_37534/3497959763.py:43: RuntimeWarning: invalid value encountered in divide
  F_scores = (1+pow(beta, 2))*(precision * recall)/(pow(beta, 2)*precision + recall)


In [20]:
results

,Dataset,Strain,strain_index,Day,bAdjustMFC,"(F score, IMMAGE)","(F score, Age)","(F score, Multivariate)","(NR rate over threshold, IMMAGE)","(NR rate over threshold, Age)","(NR rate over threshold, Multivariate)","(NR rate under threshold, IMMAGE)","(NR rate under threshold, Age)","(NR rate under threshold, Multivariate)","(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)"
6,GSE59654.SDY404,A_Perth_16_2009,0,AdjMFC,True,0.82,0.52,0.52,1.00,0.42,0.44,0.18,0.00,0.20,0.91,0.47,0.48
7,GSE59654.SDY404,A_California_7_2009,1,AdjMFC,True,0.82,0.52,0.52,1.00,0.42,0.44,0.18,0.00,0.20,0.91,0.47,0.48
8,GSE59654.SDY404,B_Brisbane_60_2008,2,AdjMFC,True,0.82,0.52,0.52,1.00,0.42,0.44,0.18,0.00,0.20,0.91,0.47,0.48
9,GSE59654.SDY404,A_Perth_16_2009,0,AdjMFC,True,0.82,0.52,0.52,1.00,0.42,0.44,0.18,0.00,0.20,0.91,0.47,0.48
10,GSE59654.SDY404,A_California_7_2009,1,AdjMFC,True,0.82,0.52,0.52,1.00,0.42,0.44,0.18,0.00,0.20,0.91,0.47,0.48
11,GSE59654.SDY404,B_Brisbane_60_2008,2,AdjMFC,True,0.82,0.52,0.52,1.00,0.42,0.44,0.18,0.00,0.20,0.91,0.47,0.48
12,GSE59743.SDY400,B_Wisonsin_01_2010,0,AdjMFC,True,0.75,0.90,1.00,0.67,1.00,1.00,0.00,0.10,0.00,0.71,0.95,1.00
13,GSE59743.SDY400,A_Victoria_361_2011,1,AdjMFC,True,0.75,0.90,1.00,0.67,1.00,1.00,0.00,0.10,0.00,0.71,0.95,1.00
14,GSE59743.SDY400,A_California_7_2009,2,AdjMFC,True,0.75,0.90,1.00,0.67,1.00,1.00,0.00,0.10,0.00,0.71,0.95,1.00
15,GSE59743.SDY400,B_Wisonsin_01_2010,0,AdjMFC,True,0.75,0.90,1.00,0.67,1.00,1.00,0.00,0.10,0.00,0.71,0.95,1.00


In [21]:
# Generate figures for promising datasets
for index, row in results.iterrows():
    parameters = {
        "bAdjustMFC" : row["bAdjustMFC"],
        "dataset_name": row[dataset_col],
        "strain_index": row[strain_index_col],
        "day": row[day_col]
    }
    print(f'exporting {row[dataset_col]}, strain no. {row[strain_index_col]}: {row[strain_col]}, day: {row[day_col]}')
    output_notebook_name = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}"
    output_notebook = f"export/{output_notebook_name}.ipynb"
    try:
            pm.execute_notebook(
                    input_path="vaccines-4.ipynb",
                    output_path=output_notebook,
                    parameters=parameters,
                    prepare_only=True
            )
    except Exception as e:
            print (f"******\nCaught exception when runnnig {output_notebook}\n******\n")
            raise(e)
    # Export the executed notebook to HTML
    output_html = f"{output_notebook_name}.html"
    os.system(f"jupyter nbconvert --execute --no-input --to html {output_notebook} --output {output_html}")

exporting GSE59654.SDY404, strain no. 0: A_Perth_16_2009, day: AdjMFC


[NbConvertApp] Converting notebook export/GSE59654.SDY404_A_Perth_16_2009_AdjMFC_discard_seroprotected_older-only.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 5 image(s).
[NbConvertApp] Writing 625542 bytes to export/GSE59654.SDY404_A_Perth_16_2009_AdjMFC_discard_seroprotected_older-only.html


exporting GSE59654.SDY404, strain no. 1: A_California_7_2009, day: AdjMFC


[NbConvertApp] Converting notebook export/GSE59654.SDY404_A_California_7_2009_AdjMFC_discard_seroprotected_older-only.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 5 image(s).
[NbConvertApp] Writing 625546 bytes to export/GSE59654.SDY404_A_California_7_2009_AdjMFC_discard_seroprotected_older-only.html


exporting GSE59654.SDY404, strain no. 2: B_Brisbane_60_2008, day: AdjMFC


...

[NbConvertApp] Converting notebook export/GSE59635.SDY63_A_California_7_2009_FC_discard_seroprotected_older-only.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 4 image(s).
[NbConvertApp] Writing 582714 bytes to export/GSE59635.SDY63_A_California_7_2009_FC_discard_seroprotected_older-only.html


exporting GSE59635.SDY63, strain no. 0: A_Perth_16_2009, day: HAI.D28


[NbConvertApp] Converting notebook export/GSE59635.SDY63_A_Perth_16_2009_HAI.D28_discard_seroprotected_older-only.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 4 image(s).
[NbConvertApp] Writing 577147 bytes to export/GSE59635.SDY63_A_Perth_16_2009_HAI.D28_discard_seroprotected_older-only.html


exporting GSE59635.SDY63, strain no. 1: B_Brisbane_60_2008, day: HAI.D28


[NbConvertApp] Converting notebook export/GSE59635.SDY63_B_Brisbane_60_2008_HAI.D28_discard_seroprotected_older-only.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 4 image(s).
[NbConvertApp] Writing 590912 bytes to export/GSE59635.SDY63_B_Brisbane_60_2008_HAI.D28_discard_seroprotected_older-only.html


In [28]:
# Save promising results to a spreadsheet
def generate_html_path(row):
    output_html = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}.html"
    output_html = os.path.join(get_dir_by_name('code'), "export", output_html)    
    output_html = '=HYPERLINK("file://' + output_html + '", "Link")'
    return output_html

results = results[cols_to_access] 
results['html_path'] = results.apply(generate_html_path, axis=1)
results.to_csv(f"export/results_{seroprotected_str}{age_restrict_str}.csv", index=False, float_format='%.2f')